In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

In [ ]:
df = pd.read_csv('~/Desktop/Projects/Project_4/Data/gemini_BTCUSD_1hr.csv', skiprows = 1)

In [ ]:
def timestamp(df):
    timestamp = pd.Timestamp(df)
    return timestamp

In [ ]:
df['Date'] = np.vectorize(timestamp)(df['Date'])

In [ ]:
df_drop = df.drop(labels= ['Unix Timestamp', 'Symbol'], axis = 1)

In [ ]:
def shorten(df):
    timestamp1 = pd.Timestamp('2019-05-01 00:00:00')
    df = df[df.Date < timestamp1]
    return df

In [ ]:
df_drop.head()

In [ ]:
df_drop = shorten(df_drop)

In [ ]:
#Add difference between high and low to capture the variance over the day
df_drop['daily_variance'] = df_drop['High'] - df_drop['Low']

In [ ]:
df_drop['daily difference'] = df_drop['Open']-df_drop['Close']

In [ ]:
df_drop['Trading_Volume'] = df_drop['Close']*df_drop['Volume']

In [ ]:
df_drop.head()

In [ ]:
def priceovertime(x, df, days, ago = True):
    price = []
    length = len(x)
    count = 0
    hours = int(days*24)
    for index, c in enumerate(x):
        if ago:
            difference = c - df['Close'].iloc[index+hours]
        else:
            difference = df['Close'].iloc[index+hours] - c
        price.append(difference)
        count+=1
        if count == (length-hours):
            if ago:
                for i in range(hours):
                    price.append(0)
            else:
                for i in range(hours):
                    price.insert(0, 0)
            break
    return price

In [ ]:
#Add price differences over time
df_drop['price_1month'] = priceovertime(x,df_drop,30)
df_drop['price_2weeks_ago'] = priceovertime(x,df_drop,14)
df_drop['price_1week_ago'] = priceovertime(x,df_drop,7)
df_drop['price_2days_ago'] = priceovertime(x,df_drop,2)
df_drop['price_1day_ago'] = priceovertime(x,df_drop,1)
df_drop['price_1hour_ago'] = priceovertime(x,df_drop,1/24)
df_drop['price_1month_ahead'] = priceovertime(x,df_drop,30,ago = False)
df_drop['price_2weeks_ahead'] = priceovertime(x,df_drop,14,ago = False)
df_drop['price_1week_ahead'] = priceovertime(x,df_drop,7,ago = False)
df_drop['price_2days_ahead'] = priceovertime(x,df_drop,2,ago = False)
df_drop['price_1day_ahead'] = priceovertime(x,df_drop,1, ago = False)
df_drop['price_1hour_ahead'] = priceovertime(x,df_drop,1/24, ago = False)

In [ ]:
df_drop.head()

In [ ]:
#Plot Volume the monetary worth of bitcoins traded over the day
plt.figure(figsize=(10,3))
plt.bar(df_drop['Date'], df_drop['Trading_Volume'])

In [ ]:
#Plot Volume of bitcoins traded over the day
plt.figure(figsize=(10,3))
plt.bar(df_drop['Date'], df_drop['Volume'])

Looking at the Graph a peak in 2017 can be observed, before th rallye to 20,000 USD began. Maybe it would have been possible to predict

In [ ]:
pkl.dump(df_drop,open('bitcoins.pkl', 'wb'))

In [ ]:
fig, axs = plt.subplots(2, 2, figsize= (25,10))
axs[0, 0].bar(df_drop['Date'], df_drop['Volume'])
axs[0, 0].set_title('Volume(BTC) traded over the day')
axs[0, 1].bar(df_drop['Date'], df_drop['Trading_Volume'])
axs[0, 1].set_title('Volume(USD) traded over the day')
axs[1, 0].plot(df_drop['Date'], df_drop['Close'])
axs[1, 0].set_title('Closing Price')
axs[1, 1].plot(df_drop['Date'], df_drop['daily_variance'])
axs[1, 1].set_title('Variance in Price over day')

In [ ]:
ls